In [9]:
import pandas as pd
import psycopg2
import os.path

from os import path

In [10]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="docker123", host="localhost", port="5432")
cur = conn.cursor()
sql = 'select * from MACHINE_ANALISE'
pdAnalise = pd.read_sql_query(sql, conn)

In [11]:
pdAnalise["FaixaRenda"] = 0
pdAnalise["FaixaEtaria"] = 0
pdAnalise["FaixaEmprestimo"] = 0

In [12]:
pdAnalise = pdAnalise[pdAnalise["IDADE"] > 0]
pdAnalise[pdAnalise["IDADE"] < 0 ]

,id,RENDA,IDADE,EMPRESTIMO,RESULTADO,FaixaRenda,FaixaEtaria,FaixaEmprestimo


In [13]:
pdAnalise.loc[(pdAnalise["RENDA"] < 20000), "FaixaRenda"] = 0
pdAnalise.loc[(pdAnalise["RENDA"] >= 20000) & (pdAnalise["RENDA"] < 30000), "FaixaRenda"] = 1
pdAnalise.loc[(pdAnalise["RENDA"] >= 30000) & (pdAnalise["RENDA"] < 40000), "FaixaRenda"] = 2
pdAnalise.loc[(pdAnalise["RENDA"] >= 40000) & (pdAnalise["RENDA"] < 50000), "FaixaRenda"] = 3
pdAnalise.loc[(pdAnalise["RENDA"] >= 50000) & (pdAnalise["RENDA"] < 60000), "FaixaRenda"] = 4
pdAnalise.loc[(pdAnalise["RENDA"] >= 60000) & (pdAnalise["RENDA"] < 70000), "FaixaRenda"] = 5
pdAnalise.loc[(pdAnalise["RENDA"] >= 70000), "FaixaRenda"] = 6

In [14]:
pdAnalise.loc[(pdAnalise["IDADE"]) >= 18 & (pdAnalise["IDADE"] < 25), "FaixaEtaria"] = 0
pdAnalise.loc[(pdAnalise["IDADE"]) >= 25 & (pdAnalise["IDADE"] < 60), "FaixaEtaria"] = 1
pdAnalise.loc[(pdAnalise["IDADE"] >= 60), "FaixaEtaria"] = 2

In [15]:
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] < 2000), "FaixaEmprestimo"] = 0
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 2000) & (pdAnalise["EMPRESTIMO"] < 4000), "FaixaEmprestimo"] = 1
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 4000) & (pdAnalise["EMPRESTIMO"] < 6000), "FaixaEmprestimo"] = 2
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 6000) & (pdAnalise["EMPRESTIMO"] < 8000), "FaixaEmprestimo"] = 3
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 8000) & (pdAnalise["EMPRESTIMO"] < 10000), "FaixaEmprestimo"] = 4
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 12000) & (pdAnalise["EMPRESTIMO"] < 14000), "FaixaEmprestimo"] = 5
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 14000) & (pdAnalise["EMPRESTIMO"] < 16000), "FaixaEmprestimo"] = 6
pdAnalise.loc[(pdAnalise["EMPRESTIMO"] >= 16000), "FaixaEmprestimo"] = 7

In [16]:
# AJUSTA PROBLEMA DE BASE DESBALANCEADA

# SEPARA O REGISTRO NOVO QUE FOI IMPORTADO
dtNovRegistroAnalise = pdAnalise[pdAnalise["id"] > 2000]
dtAntigosRegistrosAnalise = pdAnalise[pdAnalise["id"] <= 2000]

# SEPARA EM DOIS DF UM SOMENTE CLASSIFICADO COM 0 E OUTRO COM 1
dtAnaliseZero = dtAntigosRegistrosAnalise[dtAntigosRegistrosAnalise["RESULTADO"] == 0]
dtAnaliseUm = dtAntigosRegistrosAnalise[dtAntigosRegistrosAnalise["RESULTADO"] == 1]

# SHUFFLE NO DF
dtAnaliseZero = dtAnaliseZero.sample(frac=1, random_state=905).reset_index(drop=True)

# DEIXA SOMENTE 1000 REGISTROS COM RESULTADOS IGUAL A 0 PARA NÃO DEIXAR A BASE DESBALANCEADA
indexZero = dtAnaliseZero[dtAnaliseZero.index > 1000]
dtAnaliseZero = dtAnaliseZero.drop(indexZero.index, axis=0)

# UNE OS DF NOVAMENTE
dtFullTratadaAnalise = pd.concat([dtAnaliseUm, dtAnaliseZero])

# SHUFFLE NO DF
dtFullTratadaAnalise = dtFullTratadaAnalise.sample(frac=1, random_state=1).reset_index(drop=True)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.externals import joblib

from sklearn.neighbors import KNeighborsClassifier

In [32]:
#############################################################################
# KNN  ######################################################################

def GeraDump(arquivo, X_train, y_train):
        
    # PASSA OS PARAMETROS
    KNN_model = KNeighborsClassifier(n_neighbors=13)
    
    # REALIZA O FIT
    KNN_model.fit(X_train, y_train)
    
    # DUMP NO MODELO
    joblib.dump(KNN_model, arquivo)
    

In [47]:
#############################################################################
# VERIFICAR SE POSSUI JA DUMP SALVO
arquivo = "./DumpPredict.sav"

# CRIA BASE TREINO E TESTE
X_train, X_test, y_train, y_test = train_test_split(dtFullTratadaAnalise.iloc[:,5:8], dtFullTratadaAnalise["RESULTADO"], test_size=0.30, random_state=27)

if not path.exists(arquivo):
    GeraDump(arquivo, X_train, y_train)

# CLASSIFICA O NOVO REGISTRO
Loaded_model = joblib.load(arquivo)
KNN_prediction_New = Loaded_model.predict(dtNovRegistroAnalise.iloc[:,5:8])


In [51]:
KNN_prediction = Loaded_model.predict(X_test)
acuracia = round(accuracy_score(KNN_prediction, y_test) * 100,2)

resultado = [acuracia, KNN_prediction_New[0]]

In [52]:
resultado

[81.61, 0]